### **imports**

In [25]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

### **making a table of all of the stars in the txt file**

In [26]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [27]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [28]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [29]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [30]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [32]:
num_iterations = 1
empty_list = []
for i in range(70, 100):
    if i == 8:
        continue
    search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    quality = g_filter[g_filter['quality'] == 'G']
    web_format = quality[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 1 (0 - classification success, 1 - suspicious period)
class = NonVar
class_probability = 0.31


######### Lightcurve features #########

amplitude = 0.3720915952595152
cusum = 2.507219375974518
eta = 47.68043356140688
hl_amp_ratio = 0.4123331458391489
kurtosis = 0.3785190454615086
n_points = 1454
period = 0.49926372669773816
period_SNR = 47.92260996839986
period_log10FAP = -22.371206033921343
period_uncertainty = 0.00012504817166289484
phase_cusum = 2.5084195796761604
phase_eta = 39.67051138127723
phi21 = 0.5608751007724238
phi31 = -2.072787131088698
quartile31 = 0.36474159473170076
r21 = 0.16273348917520356
r31 = 0.08664489098521758
shapiro_w = 0.7332004308700562
skewness = 1.383372440538336
slope_per10 = -0.005859409577320192
slope_per90 = 0.006532906217375844
stetson_k = 0.6138520871982274
weighted_mean = 15.537046634353
weighted_std = 0.18124326935621324
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.3580223548029784
cusum = 0.17977767001586986
eta = 2.064147976728531
hl_amp_ratio = 0.3345890799199205
kurtosis = -0.7601377207217381
n_points = 841
period = 0.5518230089947184
period_SNR = 168.82672160389276
period_log10FAP = -103.62454632384717
period_uncertainty = 0.00010972381888524341
phase_cusum = 0.34495260479531725
phase_eta = 0.5387501446664957
phi21 = 0.6561842106738451
phi31 = -1.3817582038080785
quartile31 = 0.5287099074045809
r21 = 0.44328653887180886
r31 = 0.22062933527770315
shapiro_w = 0.9145179390907288
skewness = -0.6593521796728552
slope_per10 = -0.029282387823347446
slope_per90 = 0.03632932698698896
stetson_k = 0.6970513471627717
weighted_mean = 15.294123790840862
weighted_std = 0.3267282575512773
Pulled 1 of 1


...

/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_e
class_probability = 0.63


######### Lightcurve features #########

amplitude = 0.07783027923469042
cusum = 0.06582113439486433
eta = 2.48906002958717
hl_amp_ratio = 1.2011125419208235
kurtosis = -0.5025643425247459
n_points = 732
period = 0.2793993308726879
period_SNR = 183.29040071321745
period_log10FAP = -104.08017242634685
period_uncertainty = 2.8298761660272564e-05
phase_cusum = 0.25780012493093374
phase_eta = 0.6759917985286144
phi21 = -2.8769802846566317
phi31 = -5.582627498925466
quartile31 = 0.10589008686117474
r21 = 0.09794456028617173
r31 = 0.037000930006424904
shapiro_w = 0.9878688454627991
skewness = 0.10423578407398999
slope_per10 = -0.06063375973783036
slope_per90 = 0.07319245039772296
stetson_k = 0.8345970064039284
weighted_mean = 14.551215591204038
weighted_std = 0.06155665168420322
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = DSCT
class_probability = 0.74


######### Lightcurve features #########

amplitude = 0.05095385421454707
cusum = 0.10909850281033558
eta = 2.234959850337405
hl_amp_ratio = 1.0536842193665061
kurtosis = -0.9407321078538384
n_points = 905
period = 0.2094679898630978
period_SNR = 190.34367749371046
period_log10FAP = -137.55540167694485
period_uncertainty = 1.5788885865897218e-05
phase_cusum = 0.24181270361722249
phase_eta = 0.6564853157002757
phi21 = -2.129052426911325
phi31 = -2.5602016742982885
quartile31 = 0.07135303710912133
r21 = 0.028981805434785565
r31 = 0.021981032758880813
shapiro_w = 0.9787673354148865
skewness = 0.05081438956919204
slope_per10 = -0.057477513454430265
slope_per90 = 0.04884948342222958
stetson_k = 0.8774660398458016
weighted_mean = 12.064486155337203
weighted_std = 0.039730545494805355


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
